# Dimensionnement d'une cuve

In [590]:
import numpy as np
import pandas as pd
import altair as alt


## Objectif

On cherche ici à déterminer les dimensions d'une cuve de récupération d'eau de pluie telles que celle-ci puisse couvrir les besoins d'une habitation.
On dispose des données suivantes : 
- pluviométrie mensuelle moyenne ;
- consommation quotidienne moyenne.

## Méthode

On travail avec un pas de temps quotidien. 
Dans un premier temps on simule une pluviométrie quotidienne à partir des moyennes mensuelles.
Puis on considère une cuve de volume infini, qui chaque jour se remplit avec la pluie et se vide avec les consommations.
On en déduit enfin le volume minimal de la cuve pour absorber tous les besoins. 

On répète cette opération $n$ fois pour s'assurer que la cuve est suffisante en cas d'année sèche. 


## Modèle de pluviométrie

### Pluviométrie moyenne

Considérons la fiche climatique de Clermont-Ferrand. 

In [591]:
#Donnees
moyenne_clermont = pd.read_csv('../data/fiche_climat_clermont.csv', sep=',', )
donnees = pd.read_csv('../data/donnees.csv', sep=',', index_col="name")

Et visualisons-la :

In [592]:
alt.Chart(moyenne_clermont.reset_index()).mark_bar().encode(
    alt.X('mois:N', title='Mois', sort =mois),
    alt.Y('hauteur', title='Hauteur'),
    tooltip=['mois', 'hauteur']
) | alt.Chart(moyenne_clermont.reset_index()).mark_bar(color='orange').encode(
    alt.X('mois:N', title='Mois', sort=mois),
    alt.Y('nb_jours', title='Nombre de jours de pluie'),
    tooltip=['mois', 'nb_jours']
)


alt.HConcatChart(...)

In [593]:
#surface_garden = donnees.loc[donnees["name"]=="surface_garden", 'value'][0] # Pourquoi ai-je besoin du "[0]" final ? 
#surface_roof = donnees[donnees["name"]=="surface_roof"].iloc[0]#[0] # Pourquoi ai-je besoin du "[0]" final ? 
surface_roof = donnees.loc["surface_roof", "value"]
surface_garden = donnees.loc["surface_garden", "value"]

A partir de ces données, on simule une année de pluviométrie.

### Pluviométrie simulée

On simule les pluies quotidiennes à partir des données mensuelles observées. 

Le nombre de jours de pluie dans le mois suit une loi normale centrée autour du nombre moyen observé.
A partir du nombre de jours de pluie simulé et la pluviométrie moyenne observée on calcule une pluviométrie moyenne pour chaque jour de pluie simulé. La pluviométrie simulée suit une loi normale centrée autour de cette valeur. 

In [594]:
##### Definition des fonctions

## Generer k dates aléatoires distinctes entre deux bornes
def generate_random_dates(start_date, end_date, k):
   date_range = (end_date - start_date).days + 1
   random_days = np.random.choice(date_range, size=k, replace=False)
   random_dates = pd.to_datetime(start_date) + pd.to_timedelta(random_days, unit='d')
   return random_dates

## Generer k hauteurs de pluie (loi normale autour d'une hauteur moyenne)
def generate_random_rain(mean_rain:float, sigma_rain:float, k:int):
   random_rain = np.random.normal(mean_rain, sigma_rain, size=k).round(1)
   return random_rain


In [595]:
#initialisation
pluviometrie = pd.DataFrame({"precipitations" : np.zeros(365)}, index=pd.date_range("20230101", periods=365))

for month in range(len(moyenne_clermont['mois'])):
    #nombre de jours ou il pleut
    avg_days_obs = round(moyenne_clermont.loc[moyenne_clermont.index[month], 'nb_jours']) 
    avg_days = round(np.random.normal(avg_days_obs, avg_days_obs/4))
    
    # choix des jours
    start_date = pluviometrie.loc[pluviometrie.index.month == month+1].index.min()
    end_date =pluviometrie.loc[pluviometrie.index.month == month+1].index.max()
    rainy_days = generate_random_dates(start_date, end_date, avg_days)

    #hauteur tombee
    avg_rain = moyenne_clermont.loc[moyenne_clermont.index[month], 'hauteur']/avg_days
    obs_rain = generate_random_rain(avg_rain, avg_rain/4, avg_days)
    
    #remplacement des valeurs
    pluviometrie.loc[pluviometrie.index.isin(rainy_days), "precipitations"] = obs_rain

alt.Chart(pluviometrie.reset_index()).mark_bar().encode(
    x=alt.X('index:T', title="Jour de l'année"),
    y=alt.Y('precipitations:Q', title="precipitations"),
    tooltip=[alt.Tooltip("precipitations:Q")]
).properties(width=600).interactive()



alt.Chart(...)

## Consommation

### Typologie des consommations

On distingue ici trois sortes de besoins : 

* irrigation (potager)
* agricole (machines)
* domestiques

Dans un premier temps, nous ne considérerons que les besoins d'irrigation. 

### Besoins potager

Le potager à irriguer a une surface de $60 m^2$ . 

Pour simplifier les calculs, on ne prendra que trois valeurs de besoins en eau du potager : 

* D'octobre à avril le potager est au repos et n'a pas besoin d'eau.
* Entre juin et août : irrigation maximale.
* En mai et septembre, irrigation modérée (moitié) 

Pour simplifier, on considérera que l'irrigation est de $20L/m^2$ tous les trois jours *sans paillage* au plus fort de l'été. 
Le paillage divise ces besoins par trois. 



In [596]:
besoins = pd.DataFrame({"potager" : np.zeros(365)}, index=pd.date_range("20230101", periods=365))
besoins["20230501":"20230531"] = 3.33
besoins["20230901":"20230930"] = 3.33
besoins["20230601":"20230831"] = 6.67

besoins["potager_paillage"] = besoins["potager"]/3

alt.Chart(besoins.reset_index()).mark_line().encode(
    x=alt.X('index:T', title="Jour de l'année"),
    y=alt.Y('potager:Q', title="Consommation"),
    tooltip=[alt.Tooltip("conso_base:Q")]
).properties(width=600).interactive()



alt.Chart(...)

## Dimension de la cuve

Quels sont les besoins totaux ? 

In [597]:
print(np.sum(besoins['potager']*100)) 


81677.0


Le potager a donc besoin, en tout, de 82 $m^3$ au cours d'une année. 
Mais une partie de ces besoins est fournie par la pluie, lorsqu'elle tombe. 

En première approximation, considérons qu'à chaque fois qu'il pleut les besoins sont nuls pour la journée, sans influer sur ceux du lendemain. 


In [598]:
df = pd.concat([pluviometrie, besoins],axis=1)

df.loc[df['potager']>0, 'potager'] -=  df.loc[df['potager']>0, 'precipitations'] # quand il pleut les besoins sont reduits

# On cherche les endroits où il a plu davantage que les besoins.
negative_needs = df.loc[df['potager']<0, "potager"]
a=0
while(negative_needs.count()>0):
    # D'abord il n'y a plus de besoin pour le jour considere
    df.loc[df.index.isin(negative_needs.index), "potager"] = 0
    # l'excedent de pluie fournit les besoins du lendemain
    negative_needs.index = negative_needs.index + pd.Timedelta('1 day')
    df.loc[df.index.isin(negative_needs.index), "potager"] += negative_needs
    # On recalcule le nombre de jours où il a trop plu
    negative_needs = df.loc[df['potager']<0, "potager"]

alt.Chart(df.reset_index()).mark_line().encode(
    x = alt.X('index:T', title="Jour de l'année"),
    y = alt.Y('potager', title="Besoins en irrigation")
).properties(width=450).interactive()
# Problème pour le dernier jour ? pas nécessairement

#pluviometrie.loc[pluviometrie.index.isin(rainy_days), "precipitations"] = obs_rain


alt.Chart(...)

In [599]:
# A partir des besoins et consommations par unité de surface,
# On obtient les besoins et consommations pour l'ensemble du potager. 

df['sum_besoins'] = df["potager"]*surface_garden
df['sum_apports'] = df['precipitations']*surface_roof
# besoins totaux donne la totalité du besoin pour le potager, soit sur 100 m2

On accède donc, via la surface du potager et celle du toit, à l'ensemble des besoins et l'ensemble des apports quotidiens. 


### Dimension de la cuve

Simulons une cuve de volume infini, qui se remplit avec les précipitations et se vide via les utilisations, et suivons son volume. 

In [600]:
#initialisation 
cuve = pd.DataFrame({"volume" : np.zeros(365)}, index=pd.date_range("20230101", periods=365))
# remplissage au cours de l'annee
for i in range(len(cuve.index)-1):
    cuve["volume"].iloc[i+1] = cuve["volume"].iloc[i] + df['sum_apports'].iloc[i] - df['sum_besoins'].iloc[i]

alt.Chart(cuve.reset_index()).mark_line().encode(
    x=alt.X('index:T', title="Jour de l'année"), 
    y=alt.Y('volume', title="Volume de la cuve"), 
    tooltip=['volume', 'index']
).interactive()


alt.Chart(...)

La cuve, initialement vide, se remplit pendant les mois d'hiver. 
Intuitivement sur la partie 01 juin - 31 oct, il suffit de prendre la différence entre le volume minimal de la cuve sur cette période et le volume maximal _précédent_ pour être certain que la cuve peut absorber ce déficit. 

In [601]:
#volume minimal
volume_minimal = cuve["20230601":"20231031"]["volume"].min()
#index correspondant 
indexmin = cuve.loc[cuve["volume"]==volume_minimal].index[0]
indexmin
#selection de cuve avec les index qui précèdent
volume_maximal = cuve.loc["20230501":indexmin]["volume"].max()

volume_necessaire = volume_maximal-volume_minimal
volume_necessaire

1454.3999999999942

On a donc besoin d'une cuve de 9197 L. 
Vérifions le comportement d'une telle cuve. 

In [602]:
#initialisation 
cuve = pd.DataFrame({"volume" : np.zeros(365)}, index=pd.date_range("20230101", periods=365))
# remplissage au cours de l'annee
for i in range(len(cuve.index)-1):
    cuve["volume"].iloc[i+1] = min(volume_necessaire,cuve["volume"].iloc[i] + df['sum_apports'].iloc[i] - df['sum_besoins'].iloc[i])
    
alt.Chart(cuve.reset_index()).mark_line().encode(
    x=alt.X('index:T', title="Jour de l'année"), 
    y=alt.Y('volume', title="Volume de la cuve"), 
    tooltip=['volume', 'index']
).interactive()


alt.Chart(...)

In [603]:
### Sous-optimal

#Prenons d'abord une cuve vide
cuve = pd.DataFrame({"volume" : np.zeros(365)}, index=pd.date_range("20230101", periods=365))
#Prenons un volume limite pour la cuve
vol_temoin = 1000
# remplissage au cours de l'annee
for i in range(len(cuve.index)-1):
    cuve["volume"].iloc[i+1] = min(vol_temoin,cuve["volume"].iloc[i] + df['sum_apports'].iloc[i] - df['sum_besoins'].iloc[i])
#Le minimum atteint par la cuve donne le volume nécessaire
#volume minimal
volume_minimal = cuve["20230601":"20231031"]["volume"].min() # les dates peuvent changer. 
vol_necessaire = vol_temoin-volume_minimal
#Nouvelle itération 
for i in range(len(cuve.index)-1):
    cuve["volume"].iloc[i+1] = min(vol_necessaire,cuve["volume"].iloc[i] + df['sum_apports'].iloc[i] - df['sum_besoins'].iloc[i])

#Illustration
alt.Chart(cuve.reset_index()).mark_line().encode(
    x=alt.X('index:T', title="Jour de l'année"), 
    y=alt.Y('volume', title="Volume de la cuve"), 
    tooltip=['volume', 'index']
).interactive()


alt.Chart(...)

In [604]:
print(vol_necessaire)

4251.5999999999985


## PLUS ULTRA

Bouts de code à retirer et problèmes

### tentative d'appel de fonction dans un dossier parent


La prochaine étape pour se débarrasser du fichier .csv de pluviométrie quotidienne est de créer une fonction qui le fait à partir de la fiche climatique. 

In [607]:
# Il faut que tu importes depuis un endroit connu
# Ton projet définit la _library_ `recuperation_eau_pluie`,
# et l'installe automatiquement de façon éditable
# (on en discute à l'occasion si ça t'intéresse)
from recuperation_eau_pluie import test_function


ModuleNotFoundError: No module named 'recuperation_eau_pluie'